In [475]:
import pandas as pd
import numpy as np
import datetime as datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


In [278]:
df_time = pd.read_csv("hittime_fullyear.csv")
df_time = df_time.sort_values(by=['visitId', 'hitnum'])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [279]:
df_time

,fullVisitorId,visitId,timeOnSite,hittime,hitnum,pt
355344,423043652415339154,1470034812,NaN,0,1,Google
355345,8294721032567046680,1470035066,NaN,0,1,Home
355346,7718623669497357235,1470035081,40.0,0,1,Home
355347,7718623669497357235,1470035081,40.0,11332,2,Store search results
355348,7718623669497357235,1470035081,40.0,16113,3,Home
...,...,...,...,...,...,...
3432075,9952572636180683443,1501657166,21.0,0,1,Google Online Store
3432076,9952572636180683443,1501657166,21.0,20676,2,Home
3432077,3800266955937177725,1501657186,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...
3432078,6344531343964394726,1501657190,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...


# Create time on home page feature

In [280]:
df_time["totalVisitHits"] = df_time["hitnum"].groupby(df_time["visitId"]).transform('count')
df_time["timeOnPage"] = df_time.hittime.diff(periods = 1)
df_time["timeOnPage"] = df_time["timeOnPage"].shift(-1)
df_time

,fullVisitorId,visitId,timeOnSite,hittime,hitnum,pt,totalVisitHits,timeOnPage
355344,423043652415339154,1470034812,NaN,0,1,Google,1,0.0
355345,8294721032567046680,1470035066,NaN,0,1,Home,1,0.0
355346,7718623669497357235,1470035081,40.0,0,1,Home,5,11332.0
355347,7718623669497357235,1470035081,40.0,11332,2,Store search results,5,4781.0
355348,7718623669497357235,1470035081,40.0,16113,3,Home,5,19596.0
...,...,...,...,...,...,...,...,...
3432075,9952572636180683443,1501657166,21.0,0,1,Google Online Store,2,20676.0
3432076,9952572636180683443,1501657166,21.0,20676,2,Home,2,-20676.0
3432077,3800266955937177725,1501657186,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...,1,0.0
3432078,6344531343964394726,1501657190,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...,1,0.0


In [281]:
df_time.timeOnPage = np.where(df_time.timeOnPage < 0, 0, df_time.timeOnPage)
df_time

,fullVisitorId,visitId,timeOnSite,hittime,hitnum,pt,totalVisitHits,timeOnPage
355344,423043652415339154,1470034812,NaN,0,1,Google,1,0.0
355345,8294721032567046680,1470035066,NaN,0,1,Home,1,0.0
355346,7718623669497357235,1470035081,40.0,0,1,Home,5,11332.0
355347,7718623669497357235,1470035081,40.0,11332,2,Store search results,5,4781.0
355348,7718623669497357235,1470035081,40.0,16113,3,Home,5,19596.0
...,...,...,...,...,...,...,...,...
3432075,9952572636180683443,1501657166,21.0,0,1,Google Online Store,2,20676.0
3432076,9952572636180683443,1501657166,21.0,20676,2,Home,2,0.0
3432077,3800266955937177725,1501657186,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...,1,0.0
3432078,6344531343964394726,1501657190,NaN,0,1,YouTube | Shop by Brand | Google Merchandise S...,1,0.0


In [282]:
df_timeOnHome = df_time[df_time.pt == "Home"]

In [283]:
df_timeOnHome = df_timeOnHome[df_timeOnHome.hitnum == 1]

In [284]:
df_timeOnHome

,fullVisitorId,visitId,timeOnSite,hittime,hitnum,pt,totalVisitHits,timeOnPage
355345,8294721032567046680,1470035066,NaN,0,1,Home,1,0.0
355346,7718623669497357235,1470035081,40.0,0,1,Home,5,11332.0
355351,4798058133221713505,1470035161,89.0,0,1,Home,9,8177.0
355360,5076747981380011349,1470035170,NaN,0,1,Home,1,0.0
355361,143953260196124648,1470035292,NaN,0,1,Home,1,0.0
...,...,...,...,...,...,...,...,...
3432036,4865831975019962975,1501656633,NaN,0,1,Home,1,0.0
3432037,8762707685667359248,1501656678,NaN,0,1,Home,1,0.0
3432062,2182591763226364361,1501656848,NaN,0,1,Home,1,0.0
3432064,9261024702496818492,1501656924,NaN,0,1,Home,1,0.0


# merge with traffic source data and clean up

In [285]:
ts = pd.read_csv("trafficSource_fullyear2.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [286]:
ts = ts.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)

In [287]:
df_ts = ts.merge(df_timeOnHome, on = "visitId").drop(["fullVisitorId_y","timeOnSite_y"], axis = 1)

In [288]:
df_ts = df_ts.drop(["totalVisitHits", "networkDomain", "metro", "region", "timeOnSite_x",
                    "pageviews", "hits", "fullVisitorId_x", "visitId", "date", 
                   "transactionRevenue", "totalTransactionRevenue", "sessionQualityDim", "continent", "subContinent",
                   "isMobile", "hitnum", "pt", "gclId"], axis = 1)

df_ts

,visitNumber,visitStartTime,channelGrouping,transactions,referralPath,campaign,source,medium,keyword,adContent,page,slot,adNetworkType,browser,operatingSystem,deviceCategory,country,city,hittime,timeOnPage
0,2,2016-12-29 15:55:02,Referral,NaN,/,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,Chrome,Macintosh,desktop,United States,San Francisco,0,0.0
1,1,2016-12-30 02:13:22,Direct,NaN,NaN,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,Chrome,Windows,desktop,Israel,Tel Aviv-Yafo,0,0.0
2,2,2016-12-29 13:26:21,Direct,NaN,NaN,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,Safari,iOS,mobile,United States,not available in demo dataset,0,0.0
3,2,2016-12-29 13:26:21,Direct,NaN,NaN,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,Safari,iOS,mobile,United States,not available in demo dataset,0,29676.0
4,3,2016-12-29 13:26:21,Referral,NaN,/,(not set),(direct),(none),NaN,NaN,NaN,NaN,NaN,Chrome,Windows,desktop,United States,San Francisco,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286744,2,2016-11-19 04:26:23,Social,NaN,/yt/about/,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,NaN,Chrome,Windows,desktop,Romania,not available in demo dataset,0,6168.0
286745,1,2016-11-19 08:59:36,Social,NaN,/yt/about/,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,NaN,Safari,Macintosh,desktop,Indonesia,Jakarta,0,3449.0
286746,1,2016-11-19 11:58:27,Social,NaN,/yt/about/,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,NaN,Safari,Macintosh,desktop,Kosovo,not available in demo dataset,0,5988.0
286747,1,2016-11-19 21:42:43,Social,NaN,/yt/about/,(not set),youtube.com,referral,NaN,NaN,NaN,NaN,NaN,Safari,Macintosh,desktop,Vietnam,not available in demo dataset,0,6388.0


# Add DoW and ToD features

In [290]:
df_ts['dow'] = df_ts.visitStartTime.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday())
df_ts['hour'] = df_ts.visitStartTime   .apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').hour)

# Impute and drop unnecessary columns

In [291]:
x = df_ts.loc[:, df_ts.columns != 'transactions']
y = df_ts.transactions
y = np.where(y.isna(), 0, 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [292]:
for column in df_ts:
    print(df_ts[column].value_counts())

1      210469
2       34402
3       13842
4        7296
5        4457
        ...  
275         1
274         1
207         1
273         1
255         1
Name: visitNumber, Length: 296, dtype: int64
2017-04-25 14:49:35    64
2017-05-09 19:56:39    25
2017-05-17 10:29:19    25
2016-10-05 21:58:06    16
2016-12-16 02:43:27    16
                       ..
2016-10-09 02:36:53     1
2016-10-14 13:36:55     1
2017-05-16 15:46:27     1
2017-08-02 00:18:48     1
2017-05-24 05:53:16     1
Name: visitStartTime, Length: 273969, dtype: int64
Social            74981
Referral          71599
Direct            57911
Organic Search    53642
Affiliates        15707
Paid Search       10278
Display            2576
(Other)              55
Name: channelGrouping, dtype: int64
1.0     5526
2.0      154
3.0       14
4.0        6
5.0        4
6.0        2
15.0       1
12.0       1
8.0        1
Name: transactions, dtype: int64
/yt/about/                                                              66731
/       

In [293]:
nacol = pd.DataFrame(x.isna().any()).reset_index()
nacol.columns = ['column', 'Bool']
nacol = nacol[nacol.Bool == True]
nacol

,column,Bool
3,referralPath,True
7,keyword,True
8,adContent,True
9,page,True
10,slot,True
11,adNetworkType,True


In [294]:
for i in nacol.column:
    try:
        x[i] = np.where(np.isnan(x[i]),0, x[i])
    except:
        x[i] = np.where(x[i].isna(),"No", x[i])
#x.isna().any()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [295]:
x.columns

Index(['visitNumber', 'visitStartTime', 'channelGrouping', 'referralPath',
       'campaign', 'source', 'medium', 'keyword', 'adContent', 'page', 'slot',
       'adNetworkType', 'browser', 'operatingSystem', 'deviceCategory',
       'country', 'city', 'hittime', 'timeOnPage', 'dow', 'hour'],
      dtype='object')

# Label encode categoricals

In [296]:
xcat = ['channelGrouping', 'referralPath',
       'campaign', 'source', 'medium', 'keyword', 'adContent', 'page', 'slot',
       'adNetworkType', 'browser', 'operatingSystem', 'deviceCategory',
       'country', 'city','dow', 'hour']

xcat

['channelGrouping',
 'referralPath',
 'campaign',
 'source',
 'medium',
 'keyword',
 'adContent',
 'page',
 'slot',
 'adNetworkType',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'country',
 'city',
 'dow',
 'hour']

In [297]:
le = preprocessing.LabelEncoder()

for i in xcat:
    x[i] = le.fit_transform(x[i])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [298]:
x = x.drop(["visitStartTime"], axis = 1)

In [299]:
x

,visitNumber,channelGrouping,referralPath,campaign,source,medium,keyword,adContent,page,slot,adNetworkType,browser,operatingSystem,deviceCategory,country,city,hittime,timeOnPage,dow,hour
0,2,6,0,0,0,0,90,21,0,0,1,9,7,0,198,455,0,0.0,3,15
1,1,2,707,0,0,0,90,21,0,0,1,9,13,0,94,516,0,0.0,4,2
2,2,2,707,0,0,0,90,21,0,0,1,30,16,1,198,581,0,0.0,3,13
3,2,2,707,0,0,0,90,21,0,0,1,30,16,1,198,581,0,29676.0,3,13
4,3,6,0,0,0,0,90,21,0,0,1,9,13,0,198,455,0,0.0,3,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286744,2,7,636,0,183,6,90,21,0,0,1,9,13,0,155,581,0,6168.0,5,4
286745,1,7,636,0,183,6,90,21,0,0,1,30,7,0,89,228,0,3449.0,5,8
286746,1,7,636,0,183,6,90,21,0,0,1,30,7,0,102,581,0,5988.0,5,11
286747,1,7,636,0,183,6,90,21,0,0,1,30,7,0,203,581,0,6388.0,5,21


# Random Forest Classifier

In [300]:
rf = RandomForestClassifier()
rf.fit(x, y) 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [319]:
scores = cross_val_score(rf, x, y, cv=5)
scores

array([0.97802964, 0.97823888, 0.97850044, 0.97801221, 0.97769795])

# RF Confusion Matrix

In [320]:
ypred = rf.predict(x)
confusion_matrix(y, ypred)

array([[281012,     28],
       [  1267,   4442]])

# RF feature importances

In [321]:
pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False)

timeOnPage         0.462774
hour               0.166537
city               0.105656
visitNumber        0.087301
dow                0.084069
operatingSystem    0.024938
referralPath       0.020625
country            0.013854
channelGrouping    0.009500
source             0.006643
browser            0.005853
medium             0.003248
deviceCategory     0.002608
keyword            0.002520
adContent          0.001903
campaign           0.000758
adNetworkType      0.000532
slot               0.000455
page               0.000227
hittime            0.000000
dtype: float64

# RF Probabilities

In [469]:
yproba_rf = rf.predict_proba(x)
yproba_rf = yproba_rf[:,1]

In [488]:
yproba_rf[yproba_rf>=0.5] = 1
yproba_rf[yproba_rf<0.5] = 0
confusion_matrix(y, yproba_rf)

array([[280965,     75],
       [   514,   5195]])

# Logistic Regression

In [323]:
lr = LogisticRegression()
lr.fit(x, y) 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [324]:
scores = cross_val_score(lr, x, y, cv=5)
scores

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

array([0.98008718, 0.98008718, 0.98008718, 0.98008718, 0.98010427])

In [454]:
yp_lr = lr.predict_proba(x)
yp_lr = yp_lr[:,1]

In [455]:
yp_lr = (yp_lr - min(yp_lr))/(max(yp_lr) - min(yp_lr))

In [456]:
yp_lr[yp_lr>0.5] = 1
yp_lr[yp_lr<0.5] = 0

In [457]:
pd.Series(y).value_counts()

0    281040
1      5709
dtype: int64

In [460]:
confusion_matrix(y, yp_lr)

array([[280678,    362],
       [  5680,     29]])